In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import time
import pandas as pd
import glob

In [8]:
def convert_string_to_number(s):
    if 'K' in s or 'k' in s: 
        number = float(s[:-1]) * 1000 
    else: 
        number = float(s)
    
    return number


In [9]:
def convert_to_minutes(time_str): 
    hours, minutes = 0, 0
    if 'h' in time_str:
        hours = int(time_str.split('h')[0])
        time_str = time_str.split('h')[1]
    if 'm' in time_str:
        minutes = int(time_str.split('m')[0])

    total_minutes = hours * 60 + minutes
    return total_minutes


In [12]:
def get_Movie_data(movie_Genre):
    driver = webdriver.Chrome()
    # movie_Genre="western"
    driver.get('https://www.imdb.com/search/title/?title_type=feature&genres='+movie_Genre+'&release_date=2024-01-01,2024-12-31');

    driver.maximize_window()
    time.sleep(3)

    is_buttonexist=True

    while is_buttonexist:
        try:
            btnsee_more=driver.find_element(By.XPATH,"//*[@id='__next']/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button/span/span")
            ActionChains(driver).move_to_element(btnsee_more).perform()
            btnsee_more.click()
            time.sleep(7) 
            is_buttonexist=True
        except :#Exception as e: 
            # print(e)
            is_buttonexist=False 

    movies_ul=driver.find_elements(By.XPATH,"//*[@id='__next']/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li")

    movie_Name_list=[]
    movie_Genre_list=[]
    movie_Ratings_list=[]
    movie_VotecountOrg_list=[]
    movie_DurationOrg_list=[]
    movie_Votecount_list=[]
    movie_Duration_list=[]

    for movie_li in movies_ul:
        try:
            movie_Name=movie_li.find_element(By.XPATH,"./div/div/div/div[1]/div[2]/div[1]/a/h3")
            movie_Name_list.append(movie_Name.text) 
        except:
            movie_Name_list.append('') 
            
        movie_Genre_list.append(movie_Genre) 
        
        try:
            movie_Ratings=movie_li.find_element(By.XPATH,"./div/div/div/div[1]/div[2]/span/div/span/span[1]")
            movie_Ratings_list.append(movie_Ratings.text) 
        except:
            movie_Ratings_list.append(0.0) 

        try:
            movie_Votecount=movie_li.find_element(By.XPATH,"./div/div/div/div[1]/div[2]/span/div/span/span[2]")
            movie_VotecountOrg_list.append(movie_Votecount.text.replace("(","").replace(")",""))
            movie_Votecount_list.append(convert_string_to_number(movie_Votecount.text.replace("(","").replace(")","").replace(" ","")))
        except:
            movie_VotecountOrg_list.append('') 
            movie_Votecount_list.append(0) 
            
        try:
            movie_Duration=movie_li.find_element(By.XPATH,"./div/div/div/div[1]/div[2]/div[2]/span[2]")
            movie_DurationOrg_list.append(movie_Duration.text) 
            movie_Duration_list.append(convert_to_minutes(movie_Duration.text)) 
        except:
            movie_DurationOrg_list.append('') 
            movie_Duration_list.append(0) 


    dfmovie=pd.DataFrame({
        "MovieName":movie_Name_list,
        "Genre":movie_Genre_list,
        "Ratings":movie_Ratings_list,
        "VotecountOrg":movie_VotecountOrg_list,
        "DurationOrg":movie_DurationOrg_list,
        "Votecount":movie_Votecount_list,
        "Duration":movie_Duration_list
        })

    dfmovie.to_csv(movie_Genre+'.csv',index=False)
    driver.quit()


In [13]:
genres=['War','Western','Musical','Sport','History']

for i in genres:
    get_Movie_data(i.lower())

In [17]:
csv_files_path = '*.csv'

csv_files = glob.glob(csv_files_path)

dataframes = []

for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

merged_df = pd.concat(dataframes, ignore_index=True)

print(merged_df)

                          MovieName    Genre  Ratings VotecountOrg  \
0                 1. I'm Still Here  history      8.8          71K   
1                 2. Saturday Night  history      7.0          25K   
2                    3. September 5  history      7.1         9.4K   
3                 4. The Apprentice  history      7.1          39K   
4                          5. Queer  history      6.5          15K   
..                              ...      ...      ...          ...   
934  83. TV Heroes of the Wild West  western      0.0          NaN   
935                   84. Pale Moon  western      0.0          NaN   
936       85. The Rattlesnake Twins  western      0.0          NaN   
937   86. The Goods, Bads and Uglys  western      0.0          NaN   
938       87. Attack on Tranquility  western      0.0          NaN   

    DurationOrg  Votecount  Duration  
0        2h 17m    71000.0       137  
1        1h 49m    25000.0       109  
2        1h 35m     9400.0        95  
3  

In [ ]:
merged_df.info()
merged_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 939 entries, 0 to 938
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MovieName     939 non-null    object 
 1   Genre         939 non-null    object 
 2   Ratings       939 non-null    float64
 3   VotecountOrg  565 non-null    object 
 4   DurationOrg   702 non-null    object 
 5   Votecount     939 non-null    float64
 6   Duration      939 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 51.5+ KB


,Ratings,Votecount,Duration
count,939.000000,939.000000,939.000000
mean,4.049734,1992.952077,74.226837
std,3.475646,10900.995884,51.131282
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,5.400000,15.000000,89.000000
75%,7.000000,191.000000,110.000000
max,10.000000,152000.000000,210.000000


In [16]:
from sqlalchemy import create_engine

username = 'root'
password = 'abcd'
host = 'localhost'
port = '3306'
database = 'IMDB'

connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'

engine = create_engine(connection_string)

merged_df.to_sql('tblMovies', con=engine, if_exists='append', index=False)

C:\Users\Lathiga Magasri\AppData\Local\Temp\ipykernel_8528\2815262782.py:13: UserWarning: The provided table name 'tblMovies' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  merged_df.to_sql('tblMovies', con=engine, if_exists='append', index=False)


939